In [72]:
from haversine import haversine
import pandas as pd
import os

In [73]:
DF1_FILENAME = '6.Candidate groups.csv'
DF1_FILEPATH = os.path.join(os.path.dirname(os.getcwd()),'data','dataset',DF1_FILENAME)

DF2_FILENAME = '기존충전소위경도정보.csv'
DF2_FILEPATH = os.path.join(os.path.dirname(os.getcwd()),'data','dataset',DF2_FILENAME)

candidate_groups = pd.read_csv(DF1_FILEPATH, index_col=0)
existing_charging_station = pd.read_csv(DF2_FILEPATH, index_col=0)

In [74]:
candidate_groups['existing'] = 0
candidate_groups.rename(columns={'cluster':'cluster_이름'},inplace=True)
candidate_groups

,cluster_이름,xcrds,ycrds,existing
0,1000,127.191385,37.235027,0
1,1001,127.295784,37.188830,0
2,10039,127.110995,37.301494,0
3,10040,127.203855,37.144468,0
4,1005,127.258105,37.347407,0
...,...,...,...,...
5228,994,127.254190,37.248248,0
5229,995,127.204090,37.302216,0
5230,997,127.317470,37.192700,0
5231,9986,127.086575,37.330033,0


In [75]:
existing_charging_station['existing'] = 1
existing_charging_station.rename(columns={'이름':'cluster_이름','위도':'ycrds','경도':'xcrds'},inplace=True)
existing_charging_station

,cluster_이름,ycrds,xcrds,existing
0,신동백 두산위브더제니스,37.306879,127.130787,1
1,죽전역 솔하임,37.328545,127.114735,1
2,광교상현 솔하임,37.302515,127.074668,1
3,성복역 롯데캐슬 클라시엘(파크나인2차),37.319601,127.061733,1
4,까뮤이스테이트,37.335116,127.109361,1
...,...,...,...,...
1256,나인무인텔,37.208291,127.320402,1
1257,명가,37.277380,127.232383,1
1258,금화마을대우현대아파트,37.264701,127.110847,1
1259,플랫폼시티아이브오피스텔,37.298363,127.110283,1


### 거리 탐색

In [76]:
from turtle import distance

def get_min_node_idx(main_idx, main, compare_points):
    '''
    main을 기준으로 compare_porints들의 거리를 비교한 후 가장 가까운 위치의
    node행의 index를 반환
    '''
    distance_compare_list = [] # 각 main과 비교한 i 위치 거리
    hbl = (main["ycrds"][main_idx], main["xcrds"][main_idx])
    
    for i in range(compare_points.shape[0]):
        hcl = (compare_points["ycrds"][i], compare_points["xcrds"][i])
        result = haversine(hbl, hcl, unit='km')
        distance_compare_list.append(result)
        
    if min(distance_compare_list) < 1:
        min_j_idx = distance_compare_list.index(min(distance_compare_list))
    else:
        min_j_idx = -1
    return min_j_idx

In [77]:
distance_pair = {}

for i in range(existing_charging_station.shape[0]):
    distance_pair[i] = get_min_node_idx(i, existing_charging_station, candidate_groups)

In [78]:
distance_pair.values()
values = []
values = {val: key for key, val in distance_pair.items()}
result = {val: key for key, val in values.items()}

print(f'Original dict: {len(distance_pair)}')
print(f'Dict after removal: {len(result)}')

Original dict: 1261
Dict after removal: 875


### 비교한 값에 따라 기존에 설치되어 있는 충전소에서 가장 가까운 후보군을 변경

In [79]:
existing_charging_station.loc[1,['ycrds','xcrds','existing']]

ycrds        37.328545
xcrds       127.114735
existing             1
Name: 1, dtype: object

In [80]:
fin_location = candidate_groups.copy()

for i in range(len(distance_pair)):
    if distance_pair[i] != -1:
        fin_location.iloc[distance_pair[i],1:] = existing_charging_station.iloc[i,1:]

# 설치되어 있는 충전소 중 주변 후보군에 없는 경우 최종 입지에 추가
remaining_list = list(key for key, value in distance_pair.items() if value == -1)
fin_location = pd.concat([fin_location, existing_charging_station.iloc[remaining_list,:]])

In [83]:
fin_location.head(10)

,cluster_이름,xcrds,ycrds,existing
0,1000,37.235172,127.190877,1
1,1001,127.295784,37.188830,0
2,10039,127.110995,37.301494,0
3,10040,127.203855,37.144468,0
4,1005,127.258105,37.347407,0
5,1009,127.168801,37.242628,0
6,10114,37.334115,127.089069,1
7,10129,127.090842,37.323737,0
8,1013,127.223315,37.135049,0
9,1014,127.213345,37.315307,0


In [81]:
fin_location.to_csv(f'../data/dataset/7.fin_location.csv')